In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import SaliencyMapMethod
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-26 02:42:32.594585: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 02:42:32.597419: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-26 02:42:32.629967: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-26 02:42:32.630015: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-26 02:42:32.631708: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/Edge-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [4]:
x_train = x_train.values
y_train = y_train.values
x_val = x_val.values
y_val = y_val.values
x_test = x_test.values
y_test = y_test.values

In [5]:
model = joblib.load("/home/jovyan/Edge-IIoT/dl/dl_new/dnn.joblib")

2024-02-26 02:43:01.616992: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://14465aecf1f645b3ad870fe0cc0b148e: INVALID_ARGUMENT: ram://14465aecf1f645b3ad870fe0cc0b148e is a directory.


In [6]:
x_test_trimmed = x_test[20000:24000]
y_test_trimmed = y_test[20000:24000]

In [7]:
np.unique(y_test_trimmed)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14])

In [8]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=15,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [9]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.01, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_6265/1631696788.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA:   0%|          | 0/4000 [00:00<?, ?it/s]

JSMA: 100%|██████████| 4000/4000 [00:22<00:00, 175.51it/s]


In [10]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 674us/step
Accuracy: 0.27825
AUC: 0.5504878366054672

macro
Precision: 0.7382423780249172
Recall: 0.15773780736987794
F1 Score: 0.13489424720842152

weighted
Precision: 0.6603375613034231
Recall: 0.27825
F1 Score: 0.19493382366895173

Mean FNR: 0.842262192630122
Mean TNR: 0.9432378658410563
Mean FPR: 0.05676213415894371
Mean TPR: 0.15773780736987794


In [11]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.05, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_6265/2229238815.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:22<00:00, 175.44it/s]


In [12]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 714us/step
Accuracy: 0.27825
AUC: 0.5504878366054672

macro
Precision: 0.7382423780249172
Recall: 0.15773780736987794
F1 Score: 0.13489424720842152

weighted
Precision: 0.6603375613034231
Recall: 0.27825
F1 Score: 0.19493382366895173

Mean FNR: 0.842262192630122
Mean TNR: 0.9432378658410563
Mean FPR: 0.05676213415894371
Mean TPR: 0.15773780736987794


In [24]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.1, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_36324/1183469131.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:31<00:00, 125.19it/s]


In [25]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 615us/step
Accuracy: 0.27825
AUC: 0.5504878366054672

macro
Precision: 0.7382423780249172
Recall: 0.15773780736987794
F1 Score: 0.13489424720842152

weighted
Precision: 0.6603375613034231
Recall: 0.27825
F1 Score: 0.19493382366895173

Mean FNR: 0.842262192630122
Mean TNR: 0.9432378658410563
Mean FPR: 0.05676213415894371
Mean TPR: 0.15773780736987794


In [26]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.15, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_36324/1855246553.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:32<00:00, 121.60it/s]


In [27]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 613us/step
Accuracy: 0.27825
AUC: 0.5504878366054672

macro
Precision: 0.7382423780249172
Recall: 0.15773780736987794
F1 Score: 0.13489424720842152

weighted
Precision: 0.6603375613034231
Recall: 0.27825
F1 Score: 0.19493382366895173

Mean FNR: 0.842262192630122
Mean TNR: 0.9432378658410563
Mean FPR: 0.05676213415894371
Mean TPR: 0.15773780736987794


In [28]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.2, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_36324/1114785325.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:32<00:00, 123.74it/s]


In [29]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 629us/step
Accuracy: 0.27825
AUC: 0.5504878366054672

macro
Precision: 0.7382423780249172
Recall: 0.15773780736987794
F1 Score: 0.13489424720842152

weighted
Precision: 0.6603375613034231
Recall: 0.27825
F1 Score: 0.19493382366895173

Mean FNR: 0.842262192630122
Mean TNR: 0.9432378658410563
Mean FPR: 0.05676213415894371
Mean TPR: 0.15773780736987794


In [30]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.25, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_36324/2667259709.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:32<00:00, 124.57it/s]


In [31]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 609us/step
Accuracy: 0.27825
AUC: 0.5504878366054672

macro
Precision: 0.7382423780249172
Recall: 0.15773780736987794
F1 Score: 0.13489424720842152

weighted
Precision: 0.6603375613034231
Recall: 0.27825
F1 Score: 0.19493382366895173

Mean FNR: 0.842262192630122
Mean TNR: 0.9432378658410563
Mean FPR: 0.05676213415894371
Mean TPR: 0.15773780736987794


In [32]:
# Initialize the JSMA attack
attack = SaliencyMapMethod(classifier=classifier, theta=0.3, gamma = 0.01)

# Generate adversarial examples
x_test_adv = attack.generate(x=x_test_trimmed)

/tmp/ipykernel_36324/1503088926.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  x_test_adv = attack.generate(x=x_test_trimmed)
JSMA: 100%|██████████| 4000/4000 [00:32<00:00, 124.54it/s]


In [33]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

125/125 [==============================] - 0s 620us/step
Accuracy: 0.27825
AUC: 0.5504878366054672

macro
Precision: 0.7382423780249172
Recall: 0.15773780736987794
F1 Score: 0.13489424720842152

weighted
Precision: 0.6603375613034231
Recall: 0.27825
F1 Score: 0.19493382366895173

Mean FNR: 0.842262192630122
Mean TNR: 0.9432378658410563
Mean FPR: 0.05676213415894371
Mean TPR: 0.15773780736987794
